# Lesson 1: Quick tour of Agents, LangSmith

It's very quick and simple to setup and run an agent. In this lab, we'll setup an agent, trace its trajectory in LangSmith and demonstrate Studio.

#### from outline.txt (to be removed):
Highlights:
- long-running agents need ability to backup vs starting from scratch. Langgraph can provide that.
   - requires complete snapshots in state - requires 'files' be state, or files must be more sophisticated..
- fast/easy to create an agent



Lesson 0 - intro w Harrison/Lance
lesson 1 - [Slides only] overview on the common patterns of
   a. context isolation to sub agents,
   b. context offloading,
   c. todo list tool,
   d. task specific prompt eng.
      Context eng slides have many good citations for all. Plus the blog post. Should be quick.

   lesson 2 - build an agent fast with create_react_agent. Demo LangSmith, Studio in simple environment.
      notebook: 1_react_agent.ipynb

lesson 3 - todo tool / planning. Easy.
      notebook: 2_react_agent_w_todo.ipynb
      - Keep todo in state

lesson 4 - filesystem tool. Btw this is p cool and novel. It’s smtg we chatted abt. Mock filesystem in Langgraph state is nice shortcut.
      notebook: 3_react_agent_w_files.ipynb
      - Keep file system in state - use it to backtrack/restart

lesson 5 - subagent delegation. Also a bit novel how it does it; let’s supervisor define the subagent on the fly (tools, name, prompt). whereas for DR we carefully    \
              define subagent tools / system prompt, and only let supervisor define the research topic for each. This is more flexible, am curious on performance.
      notebook: 4_react_agent_w_subagents.ipynb


lesson 6 - prompting + applying to task (like DR).  But ofc general is the story here.
               Btw another little tip I’ve started doing is using a think_tool. Extremely useful.
               Claude interleaves thinking w/ tool calls. Can see it’s though process and understand failures. Super helpful.
      notebook: 5_deep_agent.ipynb


note: may renumber the notebooks to match lessons. For now, number to keep sequence.



#### Notes, this lesson.(to be removed)
This lesson is a lightweight example of create_react_agent giving time to do housekeeping, intro langsmith, studio

Todo This lesson:
- write file to studio dir
- add evalutation, add, intro,comments, diagrams etc.

In [1]:
# Load environment variables and set up auto-reload
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

In [2]:
%%writefile ../src/deep_agents_from_scratch/react_agent.py
import os
from typing import Literal
from tavily import TavilyClient
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic


# Create the Tavily search tool
@tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search using Tavily"""
    tavily_client = TavilyClient()  # Studio handles API key automatically
    search_results = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_results


# Create the model
llm = ChatAnthropic(model_name="claude-sonnet-4-20250514", max_tokens=64000)

# Create and export the agent for Studio
agent = create_react_agent(
    model=llm,
    tools=[internet_search],
    prompt="You are a helpful assistant that can search the internet. Use the internet_search tool to find information when needed."
)


Overwriting ../src/deep_agents_from_scratch/react_agent.py


In [3]:
from deep_agents_from_scratch.react_agent import agent

# Run a search example
print("Running search example...")
result = agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "What is the latest news about artificial intelligence, very briefly"
    }]
})

Running search example...


In [4]:
# Print the result
print("\n=== Search Result ===")
print(result["messages"][-1].content)



=== Search Result ===
Here are the latest AI news highlights:

**Google's AGI Push**: Google DeepMind outlined a new step toward artificial general intelligence (AGI) with a model that lets AI systems interact with realistic world simulations - seen as key to achieving human-level AI capabilities.

**OpenAI Developments**: OpenAI is making moves with GPT-5, offering ChatGPT Enterprise to federal agencies for $1, and releasing new open-source AI models. The company's valuation surge is creating millionaires among employees.

**Government AI Adoption**: The U.S. federal government is increasingly embracing ChatGPT across agencies to boost productivity and maintain technological leadership.

**Security Focus**: There's growing emphasis on AI security, with discussions around zero-trust principles for AI infrastructure and patches for critical vulnerabilities in AI model servers.

**Industry Criticism**: Some outlets are calling out "bullshit AI" - suggesting a pushback against AI hype an

```
project_root/
├─ pyproject.toml
├─ src/
│  └─ studio/
│     ├─ __init__.py
│     └─ joe.py
└─ your_script.py
```